In [ ]:
# 0) Kütüphaneler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)


In [ ]:
# Dosya yollarını senin dataset dizinine göre ayarla
files = {
    "ARP_Spoofing": "WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv",
    "Recon-Port_Scan": "WiFI_and_MQTT/attacks/CSV/test/Recon-Port_Scan_test.pcap.csv",
    "MQTT-Malformed_Data": "WiFI_and_MQTT/attacks/CSV/test/MQTT-Malformed_Data_test.pcap.csv",
    "TCP_IP-DDoS-UDP1": "WiFI_and_MQTT/attacks/CSV/test/TCP_IP-DDoS-UDP1_test.pcap.csv"
}

dfs = {}
for label, path in files.items():
    df = pd.read_csv(path)
    df["Label"] = label
    
    # inf → NaN
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
    
    # NaN doldurma (özellikle Rate için)
    if "Rate" in df.columns:
        df["Rate"] = df["Rate"].fillna(df["Rate"].median())
    
    dfs[label] = df

# Artık df1..df4 gibi kullanabilirsin
df1, df2, df3, df4 = dfs.values()


In [ ]:
for name, df in dfs.items():
    print(f"\n{name} --- İlk 5 Satır")
    display(df.head())
    print(df.info())
    display(df.describe())

In [ ]:
for name, df in dfs.items():
    num_cols = df.select_dtypes(include=[np.number]).columns
    inf_count = np.isinf(df[num_cols]).sum().sum()
    nan_count = df.isna().sum().sum()
    print(f"{name} → inf: {inf_count}, NaN: {nan_count}")


In [ ]:
def outlier_count(s):
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    return (s < lower).sum() + (s > upper).sum(), lower, upper

for name, df in dfs.items():
    if "Rate" in df.columns:
        cnt, low, up = outlier_count(df["Rate"])
        print(f"{name} → Rate Outlier: {cnt}, sınırlar=({low:.2f}, {up:.2f})")


In [ ]:
for name, df in dfs.items():
    if "Rate" in df.columns:
        fig, axes = plt.subplots(1, 2, figsize=(12,4))
        sns.histplot(df["Rate"], bins=50, kde=True, ax=axes[0])
        axes[0].set_title(f"{name} - Rate Histogram")
        sns.boxplot(x=df["Rate"], ax=axes[1])
        axes[1].set_title(f"{name} - Rate Boxplot")
        plt.show()


In [ ]:
# korelasyon matrisi
for name, df in dfs.items():
    corr = df.select_dtypes(include=[np.number]).corr()
    plt.figure(figsize=(10,7))
    sns.heatmap(corr, cmap="coolwarm", cbar=True)
    plt.title(f"{name} - Korelasyon Matrisi")
    plt.show()


In [ ]:
# scatter plot
for name, df in dfs.items():
    if "Rate" in df.columns and "Tot size" in df.columns:
        plt.figure(figsize=(6,5))
        sns.scatterplot(x="Rate", y="Tot size", data=df, alpha=0.5)
        plt.title(f"{name} - Rate vs Tot size")
        plt.show()


In [ ]:
df_all = pd.concat(dfs.values(), ignore_index=True)

plt.figure(figsize=(10,6))
sns.boxplot(x="Label", y="Rate", data=df_all)
plt.xticks(rotation=45)
plt.title("Saldırı Türlerine Göre Rate Dağılımı")
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Örnek dosya yolu (sen kendi datasetinden birini seçebilirsin)
path = "WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv"

df = pd.read_csv(path)

# 1) İlk gözlem
display(df.head())
print("-"*80)
print(df.info())
print("-"*80)
display(df.describe())

# 2) NaN ve inf kontrolü
nan_counts = df.isna().sum()
inf_counts = np.isinf(df.select_dtypes(include=[np.number])).sum()
print("\nNaN sayıları:\n", nan_counts[nan_counts>0])
print("\n∞ (inf) sayıları:\n", inf_counts[inf_counts>0])


In [ ]:
import pandas as pd
df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv")
print(df.head())

print("-"*100)

print(df.info())

print("-"*100)

print(df.describe())

print("-"*100)

print(df.isnull().sum())
print("Toplam NaN:", df.isnull().sum().sum())

print("-"*100)
df = df.dropna()
print("-"*100)
import numpy as np
print("Rate sütunundaki inf sayısı:", np.isinf(df["Rate"]).sum())
print("-"*100)
print(df.dtypes)
print("-"*100)
#
import matplotlib.pyplot as plt
import seaborn as sns

cols = ["Rate", "AVG", "IAT", "Time_To_Live"]

for col in cols:
    data = df[col].dropna()  # eksik varsa temizle
    plt.figure(figsize=(6,3))
    sns.boxplot(x=data, color="lightcoral")
    plt.title(f"Boxplot of {col}")
    plt.show()
    
print("-"*100)
def detect_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series[(series < lower) | (series > upper)]

out_rate = detect_outliers(df["Rate"])
out_avg = detect_outliers(df["AVG"])
out_iat = detect_outliers(df["IAT"])
out_ttl = detect_outliers(df["Time_To_Live"])

print("Rate outlier sayısı:", len(out_rate))
print("AVG outlier sayısı:", len(out_avg))
print("IAT outlier sayısı:", len(out_iat))
print("TTL outlier sayısı:", len(out_ttl))

print("/-"*100)

import matplotlib.pyplot as plt
import seaborn as sns

numeric_cols = ["Rate", "AVG", "IAT", "Time_To_Live"]

for col in numeric_cols:
    data = df[col].dropna()  # eksik varsa temizle
    
    # Histogram
    plt.figure(figsize=(6,4))
    plt.hist(data, bins=50, color="skyblue", edgecolor="black")
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Frekans")
    plt.show()
    
    # Boxplot
    plt.figure(figsize=(6,2))
    sns.boxplot(x=data, color="lightcoral")
    plt.title(f"Boxplot of {col}")
    plt.show()

print("-"*100)

import matplotlib.pyplot as plt

categorical_cols = ["Protocol Type", "TCP", "UDP", "ARP", "ICMP", "HTTP", "HTTPS", "DNS"]

for col in categorical_cols:
    plt.figure(figsize=(5,3))
    df[col].value_counts().plot(kind="bar", color="skyblue", edgecolor="black")
    plt.title(f"Frekans Dağılımı - {col}")
    plt.xlabel(col)
    plt.ylabel("Frekans")
    plt.show()

print("-"*100)

import seaborn as sns
import matplotlib.pyplot as plt

# Korelasyon matrisi
corr = df.corr(numeric_only=True)

plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm", annot=False, cbar=True)
plt.title("Korelasyon Matrisi (Heatmap)")
plt.show()

print("-"*100)

pairs = [("AVG","Tot size"), ("AVG","Max"), ("Rate","AVG"), ("IAT","AVG")]

for x,y in pairs:
    plt.figure(figsize=(5,4))
    sns.scatterplot(x=df[x], y=df[y], s=10, alpha=0.5)
    plt.title(f"{x} vs {y}")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()

print("-"*100)

plt.figure(figsize=(6,4))
sns.boxplot(x=df["TCP"], y=df["Rate"])
plt.title("Rate Dağılımı - TCP=0 ve TCP=1")
plt.xlabel("TCP (0=Yok, 1=Var)")
plt.ylabel("Rate")
plt.show()

print("-"*100)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.boxplot(x="Protocol Type", y="Rate", data=df)
plt.title("Benign Trafikte Rate Dağılımı - Protocol Type'a Göre")
plt.xlabel("Protocol Type (6=TCP, 17=UDP, 1=ICMP)")
plt.ylabel("Rate (paket/sn)")
plt.show()



In [ ]:
import pandas as pd
df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv")
print(df.head())

print("-"*100)

print(df.info())

print("-"*100)

print(df.describe())

print("-"*100)

print(df.isnull().sum())
print("Toplam NaN:", df.isnull().sum().sum())

print("-"*100)
df = df.dropna()
print("-"*100)
import numpy as np
print("Rate sütunundaki inf sayısı:", np.isinf(df["Rate"]).sum())
print("-"*100)
print(df.dtypes)
print("-"*100)
import matplotlib.pyplot as plt
import seaborn as sns

cols = ["Rate", "AVG", "IAT", "Time_To_Live"]

for col in cols:
    data = df[col].dropna()  # eksik varsa temizle
    plt.figure(figsize=(6,3))
    sns.boxplot(x=data, color="lightcoral")
    plt.title(f"Boxplot of {col}")
    plt.show()
    
print("-"*100)
def detect_outliers(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return series[(series < lower) | (series > upper)]

out_rate = detect_outliers(df["Rate"])
out_avg = detect_outliers(df["AVG"])
out_iat = detect_outliers(df["IAT"])

print("Rate outlier sayısı:", len(out_rate))
print("AVG outlier sayısı:", len(out_avg))
print("IAT outlier sayısı:", len(out_iat))

print("-"*100)

import matplotlib.pyplot as plt
import seaborn as sns

numeric_cols = ["Rate", "AVG", "IAT", "Time_To_Live"]

for col in numeric_cols:
    data = df[col].dropna()  # eksik varsa temizle
    
    # Histogram
    plt.figure(figsize=(6,4))
    plt.hist(data, bins=50, color="skyblue", edgecolor="black")
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Frekans")
    plt.show()
    
    # Boxplot
    plt.figure(figsize=(6,2))
    sns.boxplot(x=data, color="lightcoral")
    plt.title(f"Boxplot of {col}")
    plt.show()

print("-"*100)

import matplotlib.pyplot as plt

categorical_cols = ["Protocol Type", "TCP", "UDP", "ARP", "ICMP", "HTTP", "HTTPS", "DNS"]

for col in categorical_cols:
    plt.figure(figsize=(5,3))
    df[col].value_counts().plot(kind="bar", color="skyblue", edgecolor="black")
    plt.title(f"Frekans Dağılımı - {col}")
    plt.xlabel(col)
    plt.ylabel("Frekans")
    plt.show()

print("-"*100)

import seaborn as sns
import matplotlib.pyplot as plt

# Korelasyon matrisi
corr = df.corr(numeric_only=True)

plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm", annot=False, cbar=True)
plt.title("Korelasyon Matrisi (Heatmap)")
plt.show()

print("-"*100)

pairs = [("AVG","Tot size"), ("AVG","Max"), ("Rate","AVG"), ("IAT","AVG")]

for x,y in pairs:
    plt.figure(figsize=(5,4))
    sns.scatterplot(x=df[x], y=df[y], s=10, alpha=0.5)
    plt.title(f"{x} vs {y}")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show()

print("-"*100)

plt.figure(figsize=(6,4))
sns.boxplot(x=df["TCP"], y=df["Rate"])
plt.title("Rate Dağılımı - TCP=0 ve TCP=1")
plt.xlabel("TCP (0=Yok, 1=Var)")
plt.ylabel("Rate")
plt.show()

print("-"*100)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.boxplot(x="Protocol Type", y="Rate", data=df)
plt.title("Benign Trafikte Rate Dağılımı - Protocol Type'a Göre")
plt.xlabel("Protocol Type (6=TCP, 17=UDP, 1=ICMP)")
plt.ylabel("Rate (paket/sn)")
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dosyaları oku
df_benign = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv")
df_arp    = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv")

# İlgili kolonları seç (alan adlarında boşluk var; köşeli parantezle erişiyoruz)
cols = ["Rate", "IAT", "AVG", "Tot size"]
df_benign = df_benign[cols].copy()
df_arp    = df_arp[cols].copy()

# Sonsuzları NaN yap ve satır bazında düşür (oransal olarak çok az etki)
for d in (df_benign, df_arp):
    d.replace([np.inf, -np.inf], np.nan, inplace=True)
    d.dropna(subset=cols, inplace=True)

# Veri seti etiketleri ekle
df_benign["label"] = "Benign"
df_arp["label"]    = "ARP Spoofing"

# Tek bir DataFrame’te birleştir
df_all = pd.concat([df_benign, df_arp], ignore_index=True)

# (İsteğe bağlı) Görselleştirme için aşırı uçları kırp (görmeyi kolaylaştırır; analizi bozmamak için df_all_clip ayrı)
df_all_clip = df_all.copy()
df_all_clip["Rate"] = df_all_clip["Rate"].clip(upper=df_all["Rate"].quantile(0.99))
df_all_clip["IAT"]  = df_all_clip["IAT"].clip(upper=df_all["IAT"].quantile(0.99))
df_all_clip["AVG"]  = df_all_clip["AVG"].clip(upper=df_all["AVG"].quantile(0.99))
df_all_clip["Tot size"] = df_all_clip["Tot size"].clip(upper=df_all["Tot size"].quantile(0.99))

sns.set(style="whitegrid")

# --- Scatter 1: Rate vs IAT ---
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.scatterplot(
    data=df_all_clip,
    x="Rate", y="IAT", hue="label", s=12, alpha=0.5, edgecolor=None
)
plt.title("Rate vs IAT (Benign vs ARP Spoofing)")
plt.xlabel("Rate (pps)")
plt.ylabel("IAT (s)")
plt.legend(title="", loc="upper right")

# --- Scatter 2: AVG vs Tot size ---
plt.subplot(1,2,2)
sns.scatterplot(
    data=df_all_clip,
    x="AVG", y="Tot size", hue="label", s=12, alpha=0.5, edgecolor=None
)
plt.title("AVG vs Tot size (Benign vs ARP Spoofing)")
plt.xlabel("AVG (ortalama paket boyutu, bayt)")
plt.ylabel("Tot size (toplam boyut, bayt)")
plt.legend(title="", loc="upper left")

plt.tight_layout()
plt.show()

# (İsteğe bağlı) Log-ölçekli alternatif—uç ayrımı daha net olur:
fig, axes = plt.subplots(1, 2, figsize=(12,4))
sns.scatterplot(data=df_all, x="Rate", y="IAT", hue="label", s=10, alpha=0.4, ax=axes[0])
axes[0].set_xscale("log"); axes[0].set_yscale("log")
axes[0].set_title("Rate vs IAT (log-log)")

sns.scatterplot(data=df_all, x="AVG", y="Tot size", hue="label", s=10, alpha=0.4, ax=axes[1])
axes[1].set_xscale("log"); axes[1].set_yscale("log")
axes[1].set_title("AVG vs Tot size (log-log)")

for ax in axes:
    ax.legend(title="", loc="best")
    ax.grid(True, which="both", ls=":")

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.width", 180)
pd.set_option("display.max_columns", 50)

# ------------------------------------------------------------------
# 0) Veri setini yükle
# ------------------------------------------------------------------
df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv")

print("Veriyi Anlama ve Hazırlama (Data Understanding & Preparation)")
print("Veri setini yükleyin ve ilk gözlemleri yapın (head(), info(), describe() vb.).")
print("Veri setindeki sütunların (değişkenlerin) ne anlama geldiğini araştırın ve açıklayın.\n")

print(df.head())
print("-"*100)
print(df.info())
print("-"*100)

print("İlk (temizlenmemiş) describe()")
print(df.describe())
print("."*100, "\n")

# ------------------------------------------------------------------
# 1) Eksik / sonsuz değerleri tespit et ve yönet
# ------------------------------------------------------------------
print("Eksik verileri (NaN değerler) tespit edin. Bu eksik verilerle nasıl başa çıktığınızı açıklayın.\n")

num_cols = df.select_dtypes(include=[np.number]).columns

# a) ±inf → NaN
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)

# b) NaN sayıları ve oranları (temizlemeden önce raporlayalım)
na_counts_before = df[num_cols].isna().sum().sort_values(ascending=False)
na_ratio_before = (df[num_cols].isna().mean()*100).sort_values(ascending=False)
print("NaN sayıları (temizlemeden önce):\n", na_counts_before.head(12))
print("\nNaN oranları % (temizlemeden önce):\n", na_ratio_before.head(12))
print("-"*100)

# c) Tüm sayısal sütunlarda NaN → MEDYAN ile doldur (tek tip, güvenli ve istikrarlı)
for c in num_cols:
    if df[c].isna().any():
        df[c] = df[c].fillna(df[c].median())

# d) Son kontrol
print("Temizlik sonrası toplam NaN:", int(df.isna().sum().sum()))
print("Rate örnek istatistikleri (temizlikten sonra):\n", df['Rate'].describe())
print("-"*100, "\n")

# ------------------------------------------------------------------
# 2) Veri tipleri (ör: Protocol Type'ı kategorik olarak görmek isteyebiliriz)
# ------------------------------------------------------------------
print("Veri tiplerini kontrol edin ve anlamsal olarak hatalı olanları düzeltin.\n")
print(df.dtypes)
# İki farklı gösterim için; analizde int64 kalsın ama görselleştirmede kategori de kullanalım:
df["Protocol_Type_cat"] = df["Protocol Type"].astype("category")
print("-"*100, "\n")

# ------------------------------------------------------------------
# 3) Aykırı değer (IQR yöntemi) – seçili sütunlar
# ------------------------------------------------------------------
print("Varsa aykırı değerleri (outliers) tespit edin. Bu değerleri nasıl ele aldığınızı ve neden bu yöntemi seçtiğinizi belirtin.\n")

cols_for_outlier = ["Rate", "IAT", "AVG", "Time_To_Live"]
for col in cols_for_outlier:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col} → Aykırı değer sayısı: {len(outliers)}")
print("-"*100, "\n")

# ------------------------------------------------------------------
# 4) Univariate: histogram + boxplot (Rate, AVG, IAT)
# ------------------------------------------------------------------
print("Tek Değişkenli Analiz (Univariate Analysis)\n")

num_vars = ["Rate", "AVG", "IAT"]
for col in num_vars:
    fig = plt.figure(figsize=(12,4))
    # Histogram
    plt.subplot(1,2,1)
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f"{col} - Histogram")
    # Boxplot
    plt.subplot(1,2,2)
    sns.boxplot(x=df[col], color="lightblue")
    plt.title(f"{col} - Boxplot")
    plt.tight_layout()
    plt.show()

# TTL için ayrıca
plt.figure(figsize=(6,3))
sns.boxplot(x=df["Time_To_Live"], color="lightblue")
plt.title("Boxplot of Time_To_Live")
plt.tight_layout()
plt.show()

print("-"*100, "\n")

# ------------------------------------------------------------------
# 5) Kategorik değişkenlerin frekansı
# ------------------------------------------------------------------
print("Kategorik değişkenlerin frekanslarını bar grafikleri ile görselleştirin.\n")

plt.figure(figsize=(8,5))
sns.countplot(x="Protocol Type", data=df)
plt.title("Protocol Type - Frekans Dağılımı")
plt.tight_layout()
plt.show()

for proto in ["ICMP", "TCP", "UDP", "ARP"]:
    if proto in df.columns:
        plt.figure(figsize=(6,4))
        sns.countplot(x=proto, data=df)
        plt.title(f"{proto} - Frekans Dağılımı (0/1)")
        plt.tight_layout()
        plt.show()

print("-"*100, "\n")

# ------------------------------------------------------------------
# 6) Multivariate – Korelasyon ve scatter
# ------------------------------------------------------------------
print("Çok Değişkenli Analiz (Bivariate/Multivariate Analysis)\n")

corr = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm", annot=False, linewidths=0.5)
plt.title("Korelasyon Matrisi - Heatmap")
plt.tight_layout()
plt.show()

# Scatter örnekleri
pairs = [
    ("Rate", "Time_To_Live"),
    ("Rate", "Tot size"),
    ("AVG",  "Tot size"),
]
for x, y in pairs:
    if x in df.columns and y in df.columns:
        plt.figure(figsize=(6,4))
        sns.scatterplot(x=x, y=y, data=df, alpha=0.5)
        plt.title(f"{x} vs {y}")
        plt.tight_layout()
        plt.show()

print("-"*100, "\n")

# ------------------------------------------------------------------
# 7) Karşılaştırmalı analiz – birden fazla dosya
# ------------------------------------------------------------------
print("Karşılaştırmalı Analiz: Kategorik ve sayısal değişkenleri bir arada analiz edin.\n")

files = {
    "Benign": "WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv",
    "ARP_Spoofing": "WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv",
    "MQTT-DoS-Connect_Flood": "WiFI_and_MQTT/attacks/CSV/test/MQTT-DoS-Connect_Flood_test.pcap.csv"
}

frames = []
for label, path in files.items():
    temp = pd.read_csv(path)
    temp["Label"] = label
    num_cols_tmp = temp.select_dtypes(include=[np.number]).columns
    temp[num_cols_tmp] = temp[num_cols_tmp].replace([np.inf, -np.inf], np.nan)
    for c in num_cols_tmp:
        if temp[c].isna().any():
            temp[c] = temp[c].fillna(temp[c].median())
    frames.append(temp)

df_all = pd.concat(frames, ignore_index=True)

print(df_all["Label"].value_counts(), "\n")

plt.figure(figsize=(10,6))
sns.boxplot(x="Label", y="Rate", data=df_all)
plt.xticks(rotation=15)
plt.title("Saldırı Türüne Göre Rate Dağılımı")
plt.tight_layout()
plt.show()
